In [1]:
import tensorflow as tf
from tensorflow_model_optimization.sparsity import keras as sparsity

In [59]:
dataset = 'cifar10'
model_name = dataset + '_non-pruned'
pruned_keras_file = './models/' + dataset + '/' + model_name + '.h5'
tflite_model_file = './models/' + dataset + '/' + model_name + '.tflite'

In [60]:
#if pruned model
with sparsity.prune_scope():
    restored_model = tf.keras.models.load_model(pruned_keras_file)
#if non-pruned model
#final_model = tf.keras.models.load_model(pruned_keras_file)

In [61]:
#if pruned model
final_model = sparsity.strip_pruning(restored_model)
final_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 64)                6

In [62]:
final_model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])

In [63]:
converter = tf.lite.TFLiteConverter.from_keras_model_file(pruned_keras_file)
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_quant_model = converter.convert()
with open(tflite_model_file, 'wb') as f:
  f.write(tflite_quant_model)

INFO:tensorflow:Froze 10 variables.
INFO:tensorflow:Converted 10 variables to const ops.
